# 6. Swarm Pattern with Semantic Kernel

The swarm pattern involves multiple specialized agents that can hand off tasks to each other based on the conversation context. Each agent has specific capabilities and can transfer control to other agents when appropriate.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from datetime import date
from typing import Annotated
import json

Set up the kernel and service

In [ ]:
# Create a kernel instance
kernel = Kernel()

# Add Azure OpenAI chat completion service
service_id = "azure_openai"
kernel.add_service(
    AzureChatCompletion(
        service_id=service_id,
        deployment_name="gpt-4.1-mini",
    )
)

Define booking functions and plugins

In [ ]:
class HotelBookingPlugin:
    """
    A plugin for hotel booking operations
    """
    
    @kernel_function(
        description="Book a hotel for a given destination and number of nights",
        name="book_hotel"
    )
    def book_hotel(
        self,
        destination: Annotated[str, "The destination city for the hotel booking"],
        nights: Annotated[int, "Number of nights to stay"],
        start_date: Annotated[str, "Start date in YYYY-MM-DD format"]
    ) -> Annotated[str, "Hotel booking confirmation"]:
        return f"Hotel booked in {destination} for {nights} nights starting from {start_date}. Booking Code: HLT123"
    
    @kernel_function(
        description="Get available hotels for a given destination and start date",
        name="get_hotels"
    )
    def get_hotels(
        self,
        destination: Annotated[str, "The destination city"],
        start_date: Annotated[str, "Start date in YYYY-MM-DD format"]
    ) -> Annotated[str, "List of available hotels"]:
        hotels = ["Holiday Inn", "Marriott", "Hilton"]
        return f"Available hotels in {destination} for {start_date}: {', '.join(hotels)}"

class FlightBookingPlugin:
    """
    A plugin for flight booking operations
    """
    
    @kernel_function(
        description="Get available flights from origin to destination on a specific date",
        name="get_flights_available"
    )
    def get_flights_available(
        self,
        origin: Annotated[str, "Origin city"],
        destination: Annotated[str, "Destination city"],
        date: Annotated[str, "Flight date in YYYY-MM-DD format"]
    ) -> Annotated[str, "List of available flights"]:
        flights = ["Flight QF32", "Flight QF4", "Flight QF128"]
        return f"Available flights from {origin} to {destination} on {date}: {', '.join(flights)}"
    
    @kernel_function(
        description="Book a flight from one location to another on a specific date",
        name="book_flight"
    )
    def book_flight(
        self,
        flight_no: Annotated[str, "Flight number to book"],
        date: Annotated[str, "Flight date in YYYY-MM-DD format"]
    ) -> Annotated[str, "Flight booking confirmation"]:
        return f"Flight {flight_no} booked on {date}, Booking Code: FLY123"

class EventBookingPlugin:
    """
    A plugin for event booking operations
    """
    
    @kernel_function(
        description="Book an event ticket for a specific event on a specific date",
        name="book_event"
    )
    def book_event(
        self,
        event_name: Annotated[str, "Name of the event"],
        date: Annotated[str, "Event date in YYYY-MM-DD format"]
    ) -> Annotated[str, "Event booking confirmation"]:
        return f"Ticket booked for {event_name} on {date}. Ticket Code: EVT123"

Define handoff mechanism

In [ ]:
class HandoffPlugin:
    """
    Plugin to handle agent handoffs
    """
    
    @kernel_function(
        description="Transfer to Hotel Booking Agent for hotel-related queries",
        name="transfer_to_hotel_agent"
    )
    def transfer_to_hotel_agent(
        self,
        reason: Annotated[str, "Reason for transfer"]
    ) -> Annotated[str, "Transfer confirmation"]:
        return f"TRANSFER_TO:HotelBookingAgent:{reason}"
    
    @kernel_function(
        description="Transfer to Flight Booking Agent for flight-related queries",
        name="transfer_to_flight_agent"
    )
    def transfer_to_flight_agent(
        self,
        reason: Annotated[str, "Reason for transfer"]
    ) -> Annotated[str, "Transfer confirmation"]:
        return f"TRANSFER_TO:FlightBookingAgent:{reason}"
    
    @kernel_function(
        description="Transfer to Event Booking Agent for event-related queries",
        name="transfer_to_event_agent"
    )
    def transfer_to_event_agent(
        self,
        reason: Annotated[str, "Reason for transfer"]
    ) -> Annotated[str, "Transfer confirmation"]:
        return f"TRANSFER_TO:EventBookingAgent:{reason}"

Create specialized agents with kernels

In [ ]:
# Create specialized kernels for each agent
hotel_kernel = Kernel()
hotel_kernel.add_service(AzureChatCompletion(service_id=service_id, deployment_name="gpt-4.1-mini"))
hotel_kernel.add_plugin(HotelBookingPlugin(), plugin_name="hotel")
hotel_kernel.add_plugin(HandoffPlugin(), plugin_name="handoff")

flight_kernel = Kernel()
flight_kernel.add_service(AzureChatCompletion(service_id=service_id, deployment_name="gpt-4.1-mini"))
flight_kernel.add_plugin(FlightBookingPlugin(), plugin_name="flight")
flight_kernel.add_plugin(HandoffPlugin(), plugin_name="handoff")

event_kernel = Kernel()
event_kernel.add_service(AzureChatCompletion(service_id=service_id, deployment_name="gpt-4.1-mini"))
event_kernel.add_plugin(EventBookingPlugin(), plugin_name="event")
event_kernel.add_plugin(HandoffPlugin(), plugin_name="handoff")

# Create specialized agents
hotel_agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=hotel_kernel,
    name="HotelBookingAgent",
    instructions="""You are a hotel booking specialist. You can book hotels and check availability.
    
If a user asks about flights or events, use the transfer functions to hand off to the appropriate agent.
Always explain what you're doing when transferring.""",
    execution_settings={
        service_id: hotel_kernel.get_prompt_execution_settings_from_service_id(service_id)
    }
)
hotel_agent.execution_settings[service_id].function_choice_behavior = FunctionChoiceBehavior.Auto()

flight_agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=flight_kernel,
    name="FlightBookingAgent",
    instructions="""You are a flight booking specialist. You can book flights and check availability.
    
If a user asks about hotels or events, use the transfer functions to hand off to the appropriate agent.
Always explain what you're doing when transferring.""",
    execution_settings={
        service_id: flight_kernel.get_prompt_execution_settings_from_service_id(service_id)
    }
)
flight_agent.execution_settings[service_id].function_choice_behavior = FunctionChoiceBehavior.Auto()

event_agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=event_kernel,
    name="EventBookingAgent",
    instructions="""You are an event booking specialist. You can book event tickets.
    
If a user asks about hotels or flights, use the transfer functions to hand off to the appropriate agent.
Always explain what you're doing when transferring.""",
    execution_settings={
        service_id: event_kernel.get_prompt_execution_settings_from_service_id(service_id)
    }
)
event_agent.execution_settings[service_id].function_choice_behavior = FunctionChoiceBehavior.Auto()

Create swarm coordinator

In [ ]:
class SwarmCoordinator:
    """
    Coordinates agent handoffs in a swarm pattern
    """
    
    def __init__(self):
        self.agents = {
            "HotelBookingAgent": hotel_agent,
            "FlightBookingAgent": flight_agent,
            "EventBookingAgent": event_agent
        }
        self.current_agent = "HotelBookingAgent"  # Default starting agent
        self.conversation_history = ChatHistory()
    
    async def process_message(self, user_message: str, max_handoffs: int = 3) -> str:
        """
        Process a user message through the swarm, handling handoffs
        """
        print(f"→ User: {user_message}")
        self.conversation_history.add_user_message(user_message)
        
        handoff_count = 0
        
        while handoff_count < max_handoffs:
            print(f"→ Current Agent: {self.current_agent}")
            
            # Get response from current agent
            agent = self.agents[self.current_agent]
            
            # Create a fresh chat history with the conversation context
            agent_history = ChatHistory()
            
            # Add recent conversation context
            for message in self.conversation_history.messages[-10:]:  # Keep last 10 messages
                agent_history.add_message(message)
            
            async for response in agent.invoke(agent_history):
                response_content = str(response.content)
                
                # Check for handoff signal
                if "TRANSFER_TO:" in response_content:
                    transfer_info = response_content.split("TRANSFER_TO:")[1]
                    new_agent, reason = transfer_info.split(":", 1)
                    
                    print(f"→ Transferring to {new_agent}: {reason}")
                    self.current_agent = new_agent
                    handoff_count += 1
                    
                    # Add handoff message to history
                    self.conversation_history.add_assistant_message(f"Transferring to {new_agent} for {reason}")
                    break
                else:
                    # No handoff needed, return the response
                    self.conversation_history.add_assistant_message(response_content)
                    print(f"→ {self.current_agent}: {response_content}")
                    return response_content
        
        return f"Maximum handoffs ({max_handoffs}) reached. Please try rephrasing your request."
    
    def reset_conversation(self):
        """Reset the conversation history"""
        self.conversation_history = ChatHistory()
        self.current_agent = "HotelBookingAgent"
    
    def get_conversation_summary(self) -> str:
        """Get a summary of the conversation"""
        return f"Conversation with {len(self.conversation_history.messages)} messages. Current agent: {self.current_agent}"

Test the swarm system

In [ ]:
# Initialize the swarm
swarm = SwarmCoordinator()

# Test hotel booking
print("=== Test 1: Hotel Booking ===")
response = await swarm.process_message("I need to book a hotel in Paris for 3 nights starting 2024-06-01")
print(f"Final Response: {response}\n")

In [ ]:
# Test handoff from hotel to flight
print("=== Test 2: Handoff to Flight Agent ===")
response = await swarm.process_message("Actually, I also need to book a flight from London to Paris on 2024-06-01")
print(f"Final Response: {response}\n")

In [ ]:
# Test event booking
print("=== Test 3: Event Booking ===")
response = await swarm.process_message("I want to book tickets for a concert on 2024-06-02")
print(f"Final Response: {response}\n")

In [ ]:
# Show conversation summary
print("=== Conversation Summary ===")
print(swarm.get_conversation_summary())

print("\n=== Full Conversation History ===")
for i, message in enumerate(swarm.conversation_history.messages, 1):
    role = message.role.value.title()
    content = str(message.content)
    print(f"[{i}] {role}: {content}")

## Complex Multi-Agent Scenario

In [ ]:
# Reset and test a complex scenario
swarm.reset_conversation()

print("=== Complex Scenario: Complete Trip Planning ===")

queries = [
    "I'm planning a business trip to Tokyo",
    "I need flights from New York to Tokyo on March 15th",
    "Book me the QF32 flight",
    "Now I need a hotel in Tokyo for 5 nights starting March 15th",
    "And I want to attend a tech conference on March 17th"
]

for query in queries:
    print(f"\n--- Query: {query} ---")
    response = await swarm.process_message(query)
    print(f"Final Response: {response}")

## Key Differences from LangChain Swarm

1. **Plugin-Based Architecture**: Each agent has specialized plugins that define their capabilities, making it easy to add new functions.

2. **Explicit Handoff Mechanism**: Uses function calls to signal transfers between agents rather than complex graph routing.

3. **Conversation Context**: Maintains conversation history across agent transfers, ensuring continuity.

4. **Simplified Coordination**: The SwarmCoordinator manages agent transitions with simple string parsing instead of complex state management.

5. **Agent Isolation**: Each agent has its own kernel and plugins, providing better encapsulation and easier testing.

6. **Flexible Transfer Logic**: Agents can decide when to transfer based on conversation context rather than rigid rules.

7. **Easy Extension**: Adding new agents or capabilities is straightforward - just create new plugins and agents.

### Benefits:

- **Modularity**: Each agent is self-contained with its own capabilities
- **Flexibility**: Easy to modify transfer logic and add new agents
- **Maintainability**: Clear separation of concerns between different agents
- **Scalability**: Can easily add new specialized agents for different domains
- **Testability**: Each agent can be tested independently

### Use Cases:

- **Customer Service**: Different agents for billing, technical support, sales
- **E-commerce**: Product search, ordering, payment, shipping agents
- **Travel Planning**: Flights, hotels, car rentals, activities
- **Financial Services**: Account management, loans, investments, insurance
- **Healthcare**: Appointment scheduling, medical queries, prescription management

This approach provides a clean, scalable way to implement multi-agent systems where agents can collaborate and hand off tasks seamlessly based on their specialized capabilities."